In [44]:
# Dependencies
import json
import requests
import pandas as pd
import logging
import time

logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

# Google developer API key
from config import g_key

In [33]:
# import dataset
wine_review_df = pd.read_csv("winemag-data-130k-v2.csv")
wine_review_df['winery_search'] = wine_review_df['winery'] + " winery" + " " + wine_review_df['country']
wine_review_df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,winery_search
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,Nicosia winery Italy
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Quinta dos Avidagos winery Portugal
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Rainstorm winery US
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,St. Julian winery US
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Sweet Cheeks winery US


In [34]:
winery_list = wine_review_df.winery_search.unique()
winery_locations = {}

In [42]:
return_full_results = False

def get_googlemaps_results(winery, api_key=None, return_full_response=False):
    
    params = {"key": api_key, "input": winery, "inputtype": "textquery", "fields": "geometry"}

    # Set up your Geocoding url
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        
    # Ping google for the reuslts:
    response = requests.get(base_url, params=params)
    
    # Results will be in JSON format - convert to dict using requests functionality
    results = response.json()
    
    # if there's no results or an error, return empty results.

    if len(results['candidates']) == 0:
        output = {
            "latitude": None,
            "longitude": None,
        }
    else:    
        answer = results['candidates'][0]
        output = {
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
        }
        
    # Append some other details:    
    output['input_string'] = winery
    output['number_of_results'] = len(results['candidates'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

In [43]:
test_result = get_googlemaps_results("Quinta dos Avidagos winery", g_key, return_full_results)
test_result

{'latitude': 41.183402,
 'longitude': -7.757935499999999,
 'input_string': 'Quinta dos Avidagos winery',
 'number_of_results': 1,
 'status': 'OK'}

In [ ]:
#STILL NEED TO WRITE A FOR LOOP TO RUN THROUGH ALL THE WINERIES IN THE LIST